# Project 3

Import some libraries and the datasets

In [ ]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import sklearn.model_selection as skm
from sklearn.model_selection import cross_val_score
from sklearn.experimental import enable_halving_search_cv # because it is experimental we need this
from sklearn.model_selection import GridSearchCV, HalvingGridSearchCV
from ISLP import load_data, confusion_table
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (accuracy_score, log_loss, confusion_matrix)
from sklearn.ensemble import (RandomForestRegressor, RandomForestClassifier)
from sklearn.inspection import permutation_importance
from sklearn.tree import export_graphviz
from IPython.display import Image, Markdown
import graphviz
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statistics
import re

In [ ]:
# main dataset
data_index_2 = pd.read_csv('../Project_datasets/data_index_2.csv', quotechar="'")
data_index_2 = data_index_2.drop(columns=['Unnamed: 0'])
data_index_2.set_index(['Lon','Lat'])
# no idea what this is
#data_index_example = pd.read_csv('../Project_datasets/data_index - example.csv')
LPJ_GUESS_output = pd.read_csv('../Project_datasets/LPJ-GUESS_output_BERN1.csv')
# precipitation by day and statistics
Predaymean = pd.read_csv('../Project_datasets/Predaymean1961_1990.csv')
Predaymean_statistics = pd.read_csv('../Project_datasets/Predaymean1961_1990_statics.csv')
# maximum temperature by day and statistics
Tmaxdaymean = pd.read_csv('../Project_datasets/Tmaxdaymean1961_1990.csv')
Tmaxdaymean_statistics = pd.read_csv('../Project_datasets/Tmaxdaymean1961_1990_statics.csv')
# minimum temperature by day and statistics
Tmindaymean = pd.read_csv('../Project_datasets/Tmindaymean1961_1990.csv')
Tmindaymean_statistics = pd.read_csv('../Project_datasets/Tmindaymean1961_1990_statics.csv')
# mean temperature by day and statistics
Tmpdaymean = pd.read_csv('../Project_datasets/Tmpdaymean1961_1990.csv')
Tmpdaymean_statistics = pd.read_csv('../Project_datasets/Tmpdaymean1961_1990_statics.csv')
# shortwave radiation flux
Tswrfdaymean = pd.read_csv('../Project_datasets/Tswrfdaymean1961_1990.csv')
Tswrfdaymean_statistics = pd.read_csv('../Project_datasets/Tswrfdaymean1961_1990_statics.csv')

Text file

In [ ]:
gridlist_pan_gfed_ISO3_UN = open('../Project_datasets/gridlist_pan_gfed_ISO3_UN.txt','r')

Show the first entries

### Main Data set

Short legend about the colums: 

Lon: Longitude<br>
Lat: Latidude

stuff about the souil texture: <br>

clay: Clay in %<br>
silt: Silt in % (sedimentary deposit that is formed when a river deposits the material it is carrying)<br>
slay: slay in %(soil in eg. rainforest) <br>
sand: sand in % %<br>
orC: organic carbondioxid in %<br>
CN: Cyanide in %<br>
pH: pH<br>
cell fraction: portion of cells / organic material in soil in % <br>


Allways with season: Same as the tables below.

tmax: maximum temperatur in K <br>
tmin: min temperature in K <br>
tmp:mean temperature in K <br>
Pre: Precipitation, mm day-1 <br>
tswrf: Total shortwave radiation flux, W m-2


This is the GUESS output: 

NPP: net primary productivity (kg C m-2 year-1)<br>
SoilR: soil respiration (kg C m-2 year-1)<br>
MaxBiomeCmass: The maximum biomass from a single biome (kg C m-2)<br>
MxbiomeLAI: The maximum leaf area index from a single biome (unitless)<br>
VegC: Vegetation carbon poo (kg C m-2)l<br>
LitterC: Litter carbon pool (kg C m-2)<br>
SoilC: Soil carbon pool (kg C m-2)<br>
Biome_Cmass: The biome type based on the maximum biomass (category)<br>
Biome_LAI: The biome type based on the maximum LAI (category)<br>
Biome_obs: The observed biome type (category)<br>

Country codes

GFED-region: Global Fire Emissions Database (https://www.un-spider.org/global-fire-emissions-database-gfed) <br>
Pan_2007: Big reogion (Europa, Africa, Australia, USA, Russia, China, .... )<br>
ISO3: Abbreviation for country <br>
UN: Country code

In [ ]:
print(f'shape: {data_index_2.shape}')
data_index_2.head()

In [ ]:
with open("../Project_datasets/Readme.txt",'r') as script:
    speech = script.read().splitlines()

count = 1
for line in speech:
    if False:
        count+=1
        if count % 2 == 0: #this is the remainder operator
            print(line)
    else:
        print(line)

### LPJ_Guess_output


In [ ]:
print(f'shape: {LPJ_GUESS_output.shape}')
LPJ_GUESS_output.head()

### Pred (Precipitation, mm day-1)
 

In [ ]:
print(f'shape: {Predaymean.shape}')
Predaymean.head()

In [ ]:
print(f'shape: {Predaymean_statistics.shape}')
Predaymean_statistics.head()

### Tmp (Daily mean temperature, K)

In [ ]:
print(f'shape: {Tmpdaymean.shape}')
Tmpdaymean.head()

In [ ]:
print(f'shape: {Tmpdaymean_statistics.shape}')
Tmpdaymean_statistics.head()

### Tmax

In [ ]:
print(f'shape: {Tmaxdaymean.shape}')
Tmaxdaymean.head()

In [ ]:
print(f'shape: {Tmaxdaymean_statistics.shape}')
Tmaxdaymean_statistics.head()

### Tmin

In [ ]:
print(f'shape: {Tmindaymean.shape}')
Tmindaymean.head()

In [ ]:
print(f'shape: {Tmindaymean_statistics.shape}')
Tmindaymean_statistics.head()

### Tswrf (Total shortwave radiation flux, W m-2)

In [ ]:
print(f'shape: {Tswrfdaymean.shape}')
Tswrfdaymean.head()

In [ ]:
print(f'shape: {Tswrfdaymean_statistics.shape}')
Tswrfdaymean_statistics.head()

### County List and binome legend

In [ ]:
# Read the data from file into a list-of-lists table.
with open('../Project_datasets/gridlist_pan_gfed_ISO3_UN.txt') as file:
    datatable = [line.split() for line in file.read().splitlines()]

country_codes = pd.DataFrame(datatable, columns = ['Lon', 'Lat', 'GFED-region', 'Pan_2007', 'ISO3', 'UN'] )  
country_codes = country_codes.drop(index = 0)
country_codes.head()

In [ ]:
    
with open("../Project_datasets/legend of biomes.txt",'r') as script:
    speech = script.read().splitlines()

count = 1
for line in speech:
    if True:
        count+=1
        if count % 2 == 0: #this is the remainder operator
            print(line)
    else:
        print(line)

## Section 2: Visualisation


In [ ]:
print(data_index_2.describe())

Countplot to count the number of data for each biom: 

In [ ]:
# create lineplot 
g = sns.countplot(data_index_2, x = "Biome_obs")

Scatter plott - Biome_LAI: The biome type based on the maximum LAI (category)

In [ ]:
sns.scatterplot(data_index_2, x = "MaxBiomeLAI", y = "Biome_obs")

Scatterplott - Biome_Cmass: The biome type based on the maximum biomass (category)

In [ ]:
sns.scatterplot(data_index_2, x = "MaxBiomeCmax", y = "Biome_obs")

In [ ]:
sns.scatterplot(data_index_2, x="Lon", y="Lat", hue="Biome_obs")

 ## Some preparations

### number of samples of different biomes in different coutries

In [ ]:
countrys_sample_size = data_index_2.groupby(['ISO3', 'Biome_obs']).size()   ##### NOT TESTED
#countrys_sample_size = pd.DataFrame(countrys_sample_size)
#countrys_sample_size.columns = [ "Size"]           # Maybe solve later
countrys_sample_size.head()

In [ ]:
# Samply size for countries
countrys_sample_size.at['EGY']


## Section 3: Binary classification

We saw that `NaN` appeard somewhere in `data_index_2`. The following is to find out where it occurs.

In [ ]:
print(list(data_index_2.loc[:,data_index_2.isna().any()]))
data_index_2[data_index_2.isna().any(axis=1)]

Choose countries. Random.org gave us the biomes
- 17: desert
- 16: Arid shrub/steppe

In [ ]:
biome_list_3 = [17,16] # our chosen biomes

Countplot: 

In [ ]:
relevant_data = data_index_2.loc[data_index_2['Biome_obs'].isin(biome_list_3)]
relevant_data = relevant_data[~(relevant_data.isna().any(axis=1))] # delete rows with NaN
fig, ax = plt.subplots(figsize = (8,4))
ax = sns.countplot( relevant_data, x = 'ISO3', hue = 'Biome_obs')
ax.tick_params(axis='x', rotation=90)

Thus we choose Egypt to train and China as test.

In [ ]:
region_train_3 = data_index_2['ISO3'] == 'EGY' # egypt is the chosen one
region_test_3 = data_index_2['ISO3'] == 'CHN' # china is the chosen one

Here the action starts.

In [ ]:
# the following function generates X_test, Y_test and X_train, Y_train for the binary classification
def model_create_XY(region_train, region_test, \
                     drop_columns=[], use_columns=None, \
                        objective='Biome_obs', objective_list=None, continuous_Y=False):
      
      drop_columns += [objective] # we would not want our objective to appear in the training data

      # restrict to two biomes, restrict to country codes, remove any rows with NaN
      relevant_data = data_index_2
      if not continuous_Y:
            if not objective_list: objective_list=list(data_index_2.loc[region_train][objective].drop_duplicates())
            relevant_data = relevant_data.loc[data_index_2[objective].isin(objective_list)] # restrict to given biomes
      relevant_data = relevant_data[~(relevant_data.isna().any(axis=1))] # delete rows with NaN
      relevant_data_train = relevant_data.loc[region_train] # restrict to training country
      relevant_data_test = relevant_data.loc[region_test] # restrict to test country

      drop_columns += ['MaxBiomeLAI','Biome_obs','Biome_LAI','Biome_Cmax',
                       'Lon','Lat','Pan_2007','ISO3','UN','MaxBiomeCmax'] # drop these columns
      if use_columns:
            X_train = relevant_data_train[use_columns]
            X_test = relevant_data_test[use_columns]
      else:
            X_train = relevant_data_train.drop(columns=drop_columns)
            X_test = relevant_data_test.drop(columns=drop_columns)

      feature_names = list(X_train.columns)

      Y_train = relevant_data_train[objective]
      Y_test = relevant_data_test[objective]

      print(f"length of training data: {Y_train.shape[0]}")
      print(f"length of testing data: {Y_test.shape[0]}")

      return X_train, X_test, Y_train, Y_test, feature_names

Function for plotting (Permutation importance vs. purity importance)

In [ ]:
def plot_permutation_importance(clf, X, y, ax):
    result = permutation_importance(clf, X, y, n_repeats=10, random_state=42, n_jobs=2)
    perm_sorted_idx = result.importances_mean.argsort()

    ax.boxplot(
        result.importances[perm_sorted_idx].T,
        vert=False,
        labels=X.columns[perm_sorted_idx],
    )
    ax.axvline(x=0, color="k", linestyle="--")
    return ax

Binary classification model

In [ ]:
# the following generates, trains and evaluates the model
def model_train_evaluate(X_train, X_test, Y_train, Y_test, feature_names, \
                         hyperparameter_tuning=False):

    clf = RandomForestClassifier(random_state=0)
    
    if hyperparameter_tuning:
        X_hyper, Y_hyper = X_train, Y_train
        # X_hyper,_,Y_hyper,_ = skm.train_test_split(X_train, Y_train, train_size=300,random_state=0)
        # TODO: improve on this
        kfold3 = skm.KFold(3,
                        random_state=1,
                        shuffle=True) # let's try fewer fold to save running time

        fineness = 4 # determines how many parameters should be tested
        params = {
                'max_depth': np.linspace(5, 15, fineness).astype('int'),
                'n_estimators': np.linspace(10, 40, fineness).astype('int'), # CV number of trees, keep this low for part 4
                'min_samples_leaf': np.linspace(1, 5, fineness).astype('int'), # minimum leaf number
                }
        if False:
            rfc_gscv = GridSearchCV(clf, param_grid = params, scoring = "accuracy",
                                        cv = kfold3 )
        else:
            # Alternatively use HalvingGridSearchCV which is equally abismal in its performance
            rfc_gscv = HalvingGridSearchCV(clf, param_grid = params, scoring = "accuracy",
                                        cv = kfold3, min_resources=20, max_resources=30)
        
        # Fit the model
        model_rfc = rfc_gscv.fit(X_hyper, Y_hyper)

        # Model best estimator
        max_depths=model_rfc.best_estimator_.get_params()["max_depth"]
        max_trees= model_rfc.best_estimator_.get_params()["n_estimators"]
        min_samples_leaf= model_rfc.best_estimator_.get_params()["min_samples_leaf"]
        max_cvs= rfc_gscv.best_score_
        print("Max Depth: ", max_depths)
        print("Max Trees: ",max_trees)
        print("Min Leafs: ",min_samples_leaf)
        print("Max CV: ",max_cvs)

        clf = RandomForestClassifier(random_state=0, max_depth=max_depths, n_estimators=max_trees, min_samples_leaf=min_samples_leaf)

    clf.fit(X_train, Y_train)


    # Some performance 
    score_rfc_train = accuracy_score(Y_train, clf.predict(X_train))
    print('Accuracy of RandomForestClassifier on train data: {:.4f}'.format(score_rfc_train))
    kfold = skm.KFold(3, random_state=1, shuffle=True)
    # This becomes computationally quite expensive for large training sets
    scores_rfc_val = cross_val_score(clf, X_train, Y_train, cv=kfold)
    print("Accuracy RandomForestClassifier on cross validation: %0.4f ( %0.2f)" % (scores_rfc_val.mean(), scores_rfc_val.std()))
    score_rfc_test = accuracy_score(Y_test, clf.predict(X_test))
    print('Accuracy of RandomForestClassifier on test data: {:.4f}'.format(score_rfc_test))
    
    # confusion table
    display(Markdown('---\n The confusion table'))
    display(confusion_table(clf.predict(X_test),
                            Y_test))
    
    conftable = confusion_matrix(clf.predict(X_test), Y_test) # gives same results as confusion_table
    tp_and_fn = conftable.sum(0)
    tp_and_fp = conftable.sum(1)
    tp = conftable.diagonal()
    
    # Recall and precision
    display(Markdown('---\n Recall: proportion classified as True/ False of all True/Falls || [0,0]/ ([0,0] + [1,0]) || [1,1]/([0,1] + [1,1]'))
    recall = tp / tp_and_fn 
    print('along coulumns', '\nF:' , recall[0] ,'\nT:' , recall[1])
    display(Markdown('---\n Precission: proportion that werre True/ False of all samples predicted as True/ falls || [0,0]/ ([0,0] + [0,1]) || [1,1]/([1,0] + [1,1]'))
    precision = tp / tp_and_fp
    print('along rows','\nF:' , precision[0] ,'\nT:' , precision[1])
    
    # display importance table
    display(Markdown('---\n The feature importance'))
    feature_imp = pd.DataFrame(
    {'importance':clf.feature_importances_},
    index=feature_names)
    display(feature_imp.sort_values(by='importance', ascending=False))
    
    # Plot Impurity-based vs. Permutation importance
    mdi_importances = pd.Series(clf.feature_importances_, index=X_train.columns)
    # tree_importance_sorted_idx = np.argsort(clf.feature_importances_)
    # tree_indices = np.arange(0, len(clf.feature_importances_)) + 0.5

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 12))
    mdi_importances.sort_values().plot.barh(ax=ax1)
    ax1.set_xlabel("Gini importance")
    plot_permutation_importance(clf, X_train, Y_train, ax2)
    ax2.set_xlabel("Decrease in accuracy score")
    fig.suptitle(
        "Impurity-based vs. permutation importances on multicollinear features (train set)"
    )
    _ = fig.tight_layout()
    
    display(Markdown('---'))

def regression_train_evaluate(X_train, X_test, Y_train, Y_test, feature_names, \
                         hyperparameter_tuning=False):
    reg = RandomForestRegressor()
    reg.fit(X_train, Y_train)

    score_reg_train = reg.score(X_train, Y_train)
    print('Accuracy of RandomForestRegressor on train data: {:.4f}'.format(score_reg_train))
    # The following takes way to long for large training sets
    # kfold = skm.KFold(3, random_state=1, shuffle=True)
    # scores_reg_val = cross_val_score(reg, X_train, Y_train, cv=kfold)
    # print("Accuracy RandomForestRegressor on cross validation: %0.4f ( %0.2f)" % (scores_reg_val.mean(), scores_reg_val.std()))
    score_reg_test = reg.score(X_test, Y_test)
    print('Accuracy of RandomForestRegressor on test data: {:.4f}'.format(score_reg_test))

    # MSE
    Y_hat = reg.predict(X_test)
    mse = np.mean((Y_test - Y_hat)**2)
    print(f'MSE for training data {mse}')

    ax = subplots(figsize=(8,8))[1]
    ax.scatter(Y_hat, Y_test)

In [ ]:
# run the training data, test data and the model
def model_run(region_train, region_test, objective='Biome_obs', objective_list=None, \
                drop_columns=[], use_columns=None, \
                hyperparameter_tuning=False, continuous_Y=False):
    X_train, X_test, Y_train, Y_test, feature_names = model_create_XY(region_train, \
                        region_test, drop_columns, use_columns, objective, objective_list, continuous_Y)
    if continuous_Y:
        regression_train_evaluate(X_train, X_test, Y_train, Y_test, feature_names, \
                         hyperparameter_tuning)
    else:
        model_train_evaluate(X_train, X_test, Y_train, Y_test, feature_names, \
                         hyperparameter_tuning)

In [ ]:
model_run(region_train_3, region_test_3, objective_list=biome_list_3)

Some evaluation of the model.

And now run the model whilst removing the medians

In [ ]:
features_median = [feature_name for feature_name in list(data_index_2) if feature_name[-6:]=='Median']
print(f'We dropped the features: {features_median}')

model_run(region_train_3, region_test_3, objective_list=biome_list_3, drop_columns=features_median)

Let's see how important the weather is.

In [ ]:
features_weather = [feature_name for feature_name in list(data_index_2) if \
                   re.search('Fall|Summer|Winter|Spring',feature_name)]
print(f'We dropped the features: {features_weather}')
display(Markdown('---'))

model_run(region_train_3, region_test_3, objective_list=biome_list_3, drop_columns=features_weather.copy())

Apparently not important at all. Now lets see if the model works only with weather data.

In [ ]:
print(f'We only use the features: {features_weather}')
display(Markdown('---'))

model_run(region_train_3, region_test_3, objective_list=biome_list_3, use_columns=features_weather)

## Section 4: Multiclass classification

As regions we chose:
- China for training
- Africa for testing

In [ ]:
region_train_4 = data_index_2['Pan_2007'] == 'China'
region_test_4 = data_index_2['Pan_2007'] == 'Africa'

And now we run the simulation

In [ ]:
model_run(region_train_4, region_test_4, hyperparameter_tuning=True)

This is relatively abysmal performance. Fiddling with the hyperparameters changes a lot though the
hyperparameter optimisation is far from optimal.

In [ ]:
model_run(region_train_4, region_test_4, \
          hyperparameter_tuning=True, drop_columns=features_median)

Let's see what happens if we remove the entire weather data.

In [ ]:
model_run(region_train_4, region_test_4, \
          hyperparameter_tuning=True, drop_columns=features_weather)

Wow, our performance just improved. Let's see what happens if we use only weather data.

In [ ]:
print(features_weather)
model_run(region_train_4, region_test_4, \
          hyperparameter_tuning=True, use_columns=features_weather.copy())

Turns out the weather really does not play an important role. Let's see if the model works better at classifying 'Biome_Cmax'.

In [ ]:
model_run(region_train_4, region_test_4, objective='Biome_Cmax', hyperparameter_tuning=True)

Here we compare with LPJ-Guess.

## Section 5: Regression 

In this part we creatively use the same test and validation set as in part 4

In [ ]:
region_train_5 = region_train_4
region_test_5 = region_test_4

In [ ]:
model_run(region_train_5, region_test_5, objective='NPP', continuous_Y=True)